In [3]:
def generate_equations(shuffle=True, max_count=None):
    """
    Generates all possible math equations given the global configuration.
    If max_count is given, returns that many at most. If shuffle is True,
    the equation will be generated in random order.
    """
    # Generate all possible unique sets of numbers
    number_permutations = itertools.permutations(
        range(MIN_NUMBER, MAX_NUMBER + 1), 2
    )
    operators = ['+','-']

    # Shuffle if required. The downside is we need to convert to list first
    if shuffle:
        number_permutations = list(number_permutations)
        random.shuffle(number_permutations)

    # If a max_count is given, use itertools to only look at that many items
    if max_count is not None:
        number_permutations = itertools.islice(number_permutations, max_count)

    # Build an equation string for each and yield to caller
    for x, y in number_permutations:
        l = random.randint(0,1)
        yield '{} * x {} {} = {}'.format(x,operators[l],y,x//2+l)

In [2]:
import itertools
import random
import sympy
from sympy import solve
from sympy import Symbol

In [4]:
def equations_to_x_y(equations, n):
    """
    Given a list of equations, converts them to one-hot vectors to build
    two data matrixes x and y.
    """
    x_eq = np.zeros(
        (n, MAX_EQUATION_LENGTH, N_FEATURES), dtype=np.bool
    )
    y = np.zeros(
        (n, MAX_RESULT_LENGTH, N_FEATURES), dtype=np.bool
    )

    # Get the first n_test equations and convert to test vectors
    for i, equation in enumerate(itertools.islice(equations, n)):
        temp = equation.split('=')
        equation = temp[0] + " - " + temp[1]
        print("equation is",equation,end=" ")
        try:
            x = Symbol('x')
            temp = solve(equation,x)
            result = str(int(temp[0]))
            print("result is", result)
        except:
            continue

        # Pad the result with spaces
        result = ' ' * (MAX_RESULT_LENGTH - 1 - len(result)) + result

        # We end each sequence in a sequence-end-character:
        equation += '\0'
        result += '\0'

        for t, char in enumerate(equation):
            x_eq[i, t, CHAR_TO_INDEX[char]] = 1

        for t, char in enumerate(result):
            y[i, t, CHAR_TO_INDEX[char]] = 1

    return x_eq, y


def build_dataset():
    """
    Generates equations based on global config, splits them into train and test
    sets, and returns (x_test, y_test, x_train, y_train).
    """
    generator = generate_equations(max_count=N_EXAMPLES)

    # Split into training and test set based on SPLIT:
    n_test = round(SPLIT * N_EXAMPLES)
    n_train = N_EXAMPLES - n_test

    x_test, y_test = equations_to_x_y(generator, n_test)
    x_train, y_train = equations_to_x_y(generator, n_train)

    return x_test, y_test, x_train, y_train

In [5]:
def print_example_predictions(count, model, x_test, y_test):
    """
    Print some example predictions along with their target from the test set.
    """
    print('Examples:')

    # Pick some random indices from the test set
    prediction_indices = np.random.choice(
        x_test.shape[0], size=count, replace=False
    )
    # Get a prediction of each
    predictions = model.predict(x_test[prediction_indices, :])

    for i in range(count):
        print('{} = {}   (expected: {})'.format(
            one_hot_to_string(x_test[prediction_indices[i]]),
            one_hot_to_string(predictions[i]),
            one_hot_to_string(y_test[prediction_indices[i]]),
        ))

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, Dense, Activation
from tensorflow.keras.layers import TimeDistributed, Bidirectional
from tensorflow.keras.optimizers import Adam

def build_model():
    """
    Builds and returns the model based on the global config.
    """
    input_shape = (MAX_EQUATION_LENGTH, N_FEATURES)

    model = Sequential()

    # Encoder:
    model.add(Bidirectional(LSTM(20), input_shape=input_shape))
    # The RepeatVector-layer repeats the input n times
    model.add(RepeatVector(MAX_RESULT_LENGTH))

    # Decoder:
    model.add(Bidirectional(LSTM(20, return_sequences=True)))

    model.add(TimeDistributed(Dense(N_FEATURES)))
    model.add(Activation('softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.01),
        metrics=['accuracy'],
    )

    return model

In [7]:
import numpy as np

CHARS = [str(n) for n in range(10)] + ['+', '-', '*','/','.','x',' ', '\0']
CHAR_TO_INDEX = {i: c for c, i in enumerate(CHARS)}
INDEX_TO_CHAR = {c: i for c, i in enumerate(CHARS)}

def one_hot_to_index(vector):
    if not np.any(vector):
        return -1

    return np.argmax(vector)

def one_hot_to_char(vector):
    index = one_hot_to_index(vector)
    if index == -1:
        return ''

    return INDEX_TO_CHAR[index]

def one_hot_to_string(matrix):
    return ''.join(one_hot_to_char(vector) for vector in matrix)

In [8]:
MIN_NUMBER = 0
MAX_NUMBER = 99

MAX_N_EXAMPLES = (MAX_NUMBER - MIN_NUMBER) ** 2
N_EXAMPLES = 30000
N_FEATURES = len(CHARS)
MAX_NUMBER_LENGTH_LEFT_SIDE = len(str(MAX_NUMBER))
MAX_NUMBER_LENGTH_RIGHT_SIDE = MAX_NUMBER_LENGTH_LEFT_SIDE + 1
MAX_EQUATION_LENGTH = 21
MAX_RESULT_LENGTH = 40

SPLIT = .1
EPOCHS = 200
BATCH_SIZE = 256

RANDOM_SEED = 1

In [9]:
import random
import itertools
random.seed(RANDOM_SEED)

x_test, y_test, x_train, y_train = build_dataset()

/var/folders/k7/dvm_qjfs0wd10b19j7jnj8rw0000gn/T/ipykernel_19988/1617145673.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (n, MAX_EQUATION_LENGTH, N_FEATURES), dtype=np.bool
/var/folders/k7/dvm_qjfs0wd10b19j7jnj8rw0000gn/T/ipykernel_19988/1617145673.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (n, MAX_RESULT_LENGTH, N_FEATURES), dtype

equation is 98 * x + 94  -  49 result is 0
equation is 8 * x + 83  -  4 result is -9
equation is 88 * x + 34  -  44 result is 0
equation is 75 * x - 11  -  38 result is 0
equation is 82 * x + 49  -  41 result is 0
equation is 27 * x + 81  -  13 result is -2
equation is 13 * x - 23  -  7 result is 2
equation is 9 * x - 95  -  5 result is 11
equation is 16 * x + 28  -  8 result is -1
equation is 54 * x + 44  -  27 result is 0
equation is 67 * x + 88  -  33 result is 0
equation is 58 * x + 77  -  29 result is 0
equation is 94 * x - 49  -  48 result is 1
equation is 10 * x - 93  -  6 result is 9
equation is 83 * x - 48  -  42 result is 1
equation is 69 * x + 71  -  34 result is 0
equation is 18 * x + 33  -  9 result is -1
equation is 81 * x - 77  -  41 result is 1
equation is 68 * x + 98  -  34 result is 0
equation is 79 * x + 54  -  39 result is 0
equation is 58 * x - 72  -  30 result is 1
equation is 64 * x - 66  -  33 result is 1
equation is 17 * x - 93  -  9 result is 6
equation is 67 

equation is 95 * x + 82  -  47 result is 0
equation is 69 * x + 51  -  34 result is 0
equation is 40 * x + 12  -  20 result is 0
equation is 49 * x + 7  -  24 result is 0
equation is 14 * x - 8  -  8 result is 1
equation is 58 * x - 23  -  30 result is 0
equation is 93 * x + 11  -  46 result is 0
equation is 71 * x + 23  -  35 result is 0
equation is 10 * x - 46  -  6 result is 5
equation is 60 * x - 91  -  31 result is 2
equation is 0 * x - 15  -  1 equation is 22 * x + 2  -  11 result is 0
equation is 86 * x + 44  -  43 result is 0
equation is 72 * x + 82  -  36 result is 0
equation is 44 * x - 41  -  23 result is 1
equation is 51 * x + 68  -  25 result is 0
equation is 62 * x + 41  -  31 result is 0
equation is 30 * x + 36  -  15 result is 0
equation is 96 * x + 17  -  48 result is 0
equation is 77 * x - 12  -  39 result is 0
equation is 43 * x - 33  -  22 result is 1
equation is 91 * x + 90  -  45 result is 0
equation is 46 * x - 0  -  24 result is 0
equation is 91 * x + 65  -  45 

equation is 34 * x - 51  -  18 result is 2
equation is 13 * x + 88  -  6 result is -6
equation is 44 * x + 85  -  22 result is -1
equation is 84 * x - 73  -  43 result is 1
equation is 43 * x + 29  -  21 result is 0
equation is 64 * x - 98  -  33 result is 2
equation is 98 * x - 2  -  50 result is 0
equation is 68 * x + 91  -  34 result is 0
equation is 25 * x - 9  -  13 result is 0
equation is 97 * x + 36  -  48 result is 0
equation is 94 * x - 72  -  48 result is 1
equation is 1 * x - 88  -  1 result is 89
equation is 17 * x + 21  -  8 result is 0
equation is 24 * x + 36  -  12 result is -1
equation is 76 * x + 23  -  38 result is 0
equation is 63 * x + 7  -  31 result is 0
equation is 42 * x - 50  -  22 result is 1
equation is 97 * x + 76  -  48 result is 0
equation is 91 * x + 70  -  45 result is 0
equation is 39 * x - 7  -  20 result is 0
equation is 65 * x - 21  -  33 result is 0
equation is 78 * x - 67  -  40 result is 1
equation is 37 * x + 7  -  18 result is 0
equation is 23 *

equation is 46 * x + 10  -  23 result is 0
equation is 96 * x + 68  -  48 result is 0
equation is 46 * x + 58  -  23 result is 0
equation is 68 * x - 29  -  35 result is 0
equation is 33 * x - 67  -  17 result is 2
equation is 68 * x - 73  -  35 result is 1
equation is 99 * x + 42  -  49 result is 0
equation is 71 * x - 5  -  36 result is 0
equation is 48 * x + 36  -  24 result is 0
equation is 87 * x - 70  -  44 result is 1
equation is 40 * x - 54  -  21 result is 1
equation is 17 * x + 81  -  8 result is -4
equation is 26 * x - 27  -  14 result is 1
equation is 45 * x + 72  -  22 result is -1
equation is 50 * x - 1  -  26 result is 0
equation is 99 * x + 38  -  49 result is 0
equation is 10 * x - 24  -  6 result is 3
equation is 57 * x - 63  -  29 result is 1
equation is 71 * x + 32  -  35 result is 0
equation is 80 * x - 93  -  41 result is 1
equation is 96 * x - 91  -  49 result is 1
equation is 21 * x - 99  -  11 result is 5
equation is 39 * x + 61  -  19 result is -1
equation is 

...

equation is 8 * x - 29  -  5 result is 4
equation is 68 * x - 3  -  35 result is 0
equation is 98 * x + 50  -  49 result is 0
equation is 18 * x - 66  -  10 result is 4
equation is 22 * x - 7  -  12 result is 0
equation is 36 * x - 60  -  19 result is 2
equation is 41 * x + 39  -  20 result is 0
equation is 24 * x + 55  -  12 result is -1
equation is 4 * x + 81  -  2 result is -19
equation is 27 * x + 77  -  13 result is -2
equation is 83 * x - 99  -  42 result is 1
equation is 88 * x - 6  -  45 result is 0
equation is 56 * x + 33  -  28 result is 0
equation is 76 * x + 88  -  38 result is 0
equation is 50 * x + 87  -  25 result is -1
equation is 66 * x + 61  -  33 result is 0
equation is 61 * x + 25  -  30 result is 0
equation is 59 * x - 84  -  30 result is 1
equation is 91 * x + 20  -  45 result is 0
equation is 91 * x + 24  -  45 result is 0
equation is 41 * x - 69  -  21 result is 2
equation is 81 * x + 44  -  40 result is 0
equation is 86 * x - 66  -  44 result is 1
equation is 8

equation is 66 * x + 31  -  33 result is 0
equation is 51 * x + 26  -  25 result is 0
equation is 59 * x + 76  -  29 result is 0
equation is 72 * x - 93  -  37 result is 1
equation is 22 * x + 85  -  11 result is -3
equation is 22 * x - 94  -  12 result is 4
equation is 26 * x - 89  -  14 result is 3
equation is 87 * x + 14  -  43 result is 0
equation is 76 * x + 2  -  38 result is 0
equation is 43 * x + 35  -  21 result is 0
equation is 24 * x + 52  -  12 result is -1
equation is 73 * x - 40  -  37 result is 1
equation is 27 * x - 41  -  14 result is 2
equation is 12 * x + 0  -  6 result is 0
equation is 31 * x + 43  -  15 result is 0
equation is 93 * x + 25  -  46 result is 0
equation is 41 * x - 11  -  21 result is 0
equation is 71 * x - 45  -  36 result is 1
equation is 52 * x - 38  -  27 result is 1
equation is 66 * x + 62  -  33 result is 0
equation is 5 * x - 6  -  3 result is 1
equation is 40 * x + 42  -  20 result is 0
equation is 42 * x + 82  -  21 result is -1
equation is 36

equation is 81 * x + 93  -  40 result is 0
equation is 51 * x + 79  -  25 result is -1
equation is 39 * x + 45  -  19 result is 0
equation is 37 * x + 35  -  18 result is 0
equation is 72 * x + 53  -  36 result is 0
equation is 82 * x - 89  -  42 result is 1
equation is 52 * x + 49  -  26 result is 0
equation is 87 * x - 49  -  44 result is 1
equation is 29 * x + 39  -  14 result is 0
equation is 1 * x + 24  -  0 result is -24
equation is 80 * x + 42  -  40 result is 0
equation is 38 * x - 65  -  20 result is 2
equation is 14 * x + 77  -  7 result is -5
equation is 28 * x - 49  -  15 result is 2
equation is 66 * x - 37  -  34 result is 1
equation is 93 * x - 34  -  47 result is 0
equation is 65 * x + 83  -  32 result is 0
equation is 45 * x + 82  -  22 result is -1
equation is 42 * x + 88  -  21 result is -1
equation is 22 * x - 92  -  12 result is 4
equation is 20 * x - 71  -  11 result is 4
equation is 22 * x - 88  -  12 result is 4
equation is 58 * x - 74  -  30 result is 1
equation

equation is 27 * x - 56  -  14 result is 2
equation is 28 * x + 8  -  14 result is 0
equation is 22 * x - 1  -  12 result is 0
equation is 14 * x - 1  -  8 result is 0
equation is 6 * x - 68  -  4 result is 12
equation is 10 * x - 81  -  6 result is 8
equation is 38 * x + 83  -  19 result is -1
equation is 88 * x - 12  -  45 result is 0
equation is 80 * x + 18  -  40 result is 0
equation is 88 * x + 42  -  44 result is 0
equation is 99 * x + 1  -  49 result is 0
equation is 90 * x - 62  -  46 result is 1
equation is 62 * x + 76  -  31 result is 0
equation is 15 * x - 95  -  8 result is 6
equation is 44 * x - 10  -  23 result is 0
equation is 35 * x - 25  -  18 result is 1
equation is 71 * x - 3  -  36 result is 0
equation is 56 * x - 11  -  29 result is 0
equation is 22 * x - 37  -  12 result is 2
equation is 93 * x - 22  -  47 result is 0
equation is 54 * x + 23  -  27 result is 0
equation is 33 * x - 23  -  17 result is 1
equation is 25 * x + 97  -  12 result is -3
equation is 2 * x 

equation is 87 * x - 31  -  44 result is 0
equation is 4 * x - 80  -  3 result is 20
equation is 69 * x - 85  -  35 result is 1
equation is 35 * x + 84  -  17 result is -1
equation is 63 * x - 8  -  32 result is 0
equation is 1 * x + 52  -  0 result is -52
equation is 89 * x + 59  -  44 result is 0
equation is 4 * x - 21  -  3 result is 6
equation is 3 * x + 69  -  1 result is -22
equation is 5 * x - 7  -  3 result is 2
equation is 52 * x + 53  -  26 result is 0
equation is 16 * x - 91  -  9 result is 6
equation is 97 * x - 82  -  49 result is 1
equation is 37 * x + 86  -  18 result is -1
equation is 44 * x - 7  -  23 result is 0
equation is 73 * x + 70  -  36 result is 0
equation is 0 * x + 35  -  0 equation is 71 * x + 61  -  35 result is 0
equation is 64 * x - 50  -  33 result is 1
equation is 4 * x - 69  -  3 result is 18
equation is 80 * x + 73  -  40 result is 0
equation is 15 * x - 53  -  8 result is 4
equation is 34 * x - 74  -  18 result is 2
equation is 62 * x - 82  -  32 res

In [10]:
import os
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

model = build_model()

model.summary()
print()

# Let's print some predictions now to get a feeling for the equations
print()
print_example_predictions(5, model, x_test, y_test)
print()

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

try:
    model.fit(
        x_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(x_test, y_test),
        verbose=2,
        callbacks=[
            ModelCheckpoint(
                'model2.h5',
                save_best_only=True,
            ),
            TensorBoard(logdir, histogram_freq=1)
        ]
    )
except KeyboardInterrupt:
    print('\nCaught SIGINT\n')

# Load weights achieving best val_loss from training:
model.load_weights('model2.h5')

print_example_predictions(20, model, x_test, y_test)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 40)               6240      
 l)                                                              
                                                                 
 repeat_vector (RepeatVector  (None, 40, 40)           0         
 )                                                               
                                                                 
 bidirectional_1 (Bidirectio  (None, 40, 40)           9760      
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 40, 18)           738       
 ibuted)                                                         
                                                                 
 activation (Activation)     (None, 40, 18)            0

/Users/nehal/Library/Python/3.8/lib/python/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-05-08 21:37:10.666268: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz




Examples:
32 * x + 11  -  16  = ++++333333333333333333333333333333333444   (expected:                                       0 )
99 * x + 68  -  49  = 0000000000000000000000000000000000000000   (expected:                                       0 )
2 * x - 93  -  2  = ........6666666666666666..........666444   (expected:                                      47 )
72 * x - 18  -  37  = ***....................................1   (expected:                                       0 )
68 * x + 1  -  34  = +++................................44444   (expected:                                       0 )

Epoch 1/200
106/106 - 6s - loss: 0.0782 - accuracy: 0.2443 - val_loss: 0.1626 - val_accuracy: 0.9362 - 6s/epoch - 57ms/step
Epoch 2/200
106/106 - 3s - loss: 0.0368 - accuracy: 0.2399 - val_loss: 0.1310 - val_accuracy: 0.9610 - 3s/epoch - 26ms/step
Epoch 3/200
106/106 - 3s - loss: 0.0373 - accuracy: 0.2439 - val_loss: 0.1973 - val_accuracy: 0.9362 - 3s/epoch - 27ms/step
Epoch 4/200
106/106 - 3s - l

Epoch 62/200
106/106 - 4s - loss: 0.0107 - accuracy: 0.2512 - val_loss: 0.0427 - val_accuracy: 0.9848 - 4s/epoch - 33ms/step
Epoch 63/200
106/106 - 3s - loss: 0.0106 - accuracy: 0.2513 - val_loss: 0.0433 - val_accuracy: 0.9845 - 3s/epoch - 31ms/step
Epoch 64/200
106/106 - 3s - loss: 0.0107 - accuracy: 0.2513 - val_loss: 0.0429 - val_accuracy: 0.9847 - 3s/epoch - 29ms/step
Epoch 65/200
106/106 - 3s - loss: 0.0106 - accuracy: 0.2513 - val_loss: 0.0424 - val_accuracy: 0.9848 - 3s/epoch - 29ms/step
Epoch 66/200
106/106 - 3s - loss: 0.0105 - accuracy: 0.2513 - val_loss: 0.0428 - val_accuracy: 0.9846 - 3s/epoch - 29ms/step
Epoch 67/200
106/106 - 3s - loss: 0.0105 - accuracy: 0.2513 - val_loss: 0.0425 - val_accuracy: 0.9845 - 3s/epoch - 29ms/step
Epoch 68/200
106/106 - 3s - loss: 0.0105 - accuracy: 0.2513 - val_loss: 0.0435 - val_accuracy: 0.9841 - 3s/epoch - 29ms/step
Epoch 69/200
106/106 - 3s - loss: 0.0103 - accuracy: 0.2513 - val_loss: 0.0413 - val_accuracy: 0.9847 - 3s/epoch - 30ms/step


Epoch 128/200
106/106 - 3s - loss: 0.0094 - accuracy: 0.2516 - val_loss: 0.0400 - val_accuracy: 0.9854 - 3s/epoch - 31ms/step
Epoch 129/200
106/106 - 3s - loss: 0.0094 - accuracy: 0.2516 - val_loss: 0.0381 - val_accuracy: 0.9862 - 3s/epoch - 32ms/step
Epoch 130/200
106/106 - 3s - loss: 0.0093 - accuracy: 0.2516 - val_loss: 0.0500 - val_accuracy: 0.9831 - 3s/epoch - 32ms/step
Epoch 131/200
106/106 - 3s - loss: 0.0475 - accuracy: 0.2423 - val_loss: 0.1303 - val_accuracy: 0.9611 - 3s/epoch - 32ms/step
Epoch 132/200
106/106 - 4s - loss: 0.0290 - accuracy: 0.2456 - val_loss: 0.1023 - val_accuracy: 0.9668 - 4s/epoch - 33ms/step
Epoch 133/200
106/106 - 4s - loss: 0.0251 - accuracy: 0.2475 - val_loss: 0.0894 - val_accuracy: 0.9711 - 4s/epoch - 33ms/step
Epoch 134/200
106/106 - 4s - loss: 0.0214 - accuracy: 0.2477 - val_loss: 0.0817 - val_accuracy: 0.9724 - 4s/epoch - 33ms/step
Epoch 135/200
106/106 - 3s - loss: 0.0200 - accuracy: 0.2484 - val_loss: 0.0776 - val_accuracy: 0.9745 - 3s/epoch - 31

106/106 - 5s - loss: 0.0126 - accuracy: 0.2508 - val_loss: 0.0502 - val_accuracy: 0.9826 - 5s/epoch - 46ms/step
Epoch 194/200
106/106 - 5s - loss: 0.0123 - accuracy: 0.2509 - val_loss: 0.0499 - val_accuracy: 0.9827 - 5s/epoch - 46ms/step
Epoch 195/200
106/106 - 5s - loss: 0.0122 - accuracy: 0.2509 - val_loss: 0.0493 - val_accuracy: 0.9830 - 5s/epoch - 46ms/step
Epoch 196/200
106/106 - 5s - loss: 0.0121 - accuracy: 0.2509 - val_loss: 0.0488 - val_accuracy: 0.9830 - 5s/epoch - 47ms/step
Epoch 197/200
106/106 - 5s - loss: 0.0121 - accuracy: 0.2508 - val_loss: 0.0481 - val_accuracy: 0.9831 - 5s/epoch - 46ms/step
Epoch 198/200
106/106 - 5s - loss: 0.0119 - accuracy: 0.2509 - val_loss: 0.0479 - val_accuracy: 0.9829 - 5s/epoch - 46ms/step
Epoch 199/200
106/106 - 5s - loss: 0.0117 - accuracy: 0.2509 - val_loss: 0.0475 - val_accuracy: 0.9830 - 5s/epoch - 46ms/step
Epoch 200/200
106/106 - 5s - loss: 0.0119 - accuracy: 0.2508 - val_loss: 0.0479 - val_accuracy: 0.9827 - 5s/epoch - 47ms/step
Exampl

In [11]:
def predict(model, equation):
    """
    Given a model and an equation string, returns the predicted result.
    """
    x = np.zeros((1, MAX_EQUATION_LENGTH, N_FEATURES), dtype=np.bool)
    equation += '\0'

    for t, char in enumerate(equation):
        x[0, t, CHAR_TO_INDEX[char]] = 1

    predictions = model.predict(x)
    return one_hot_to_string(predictions[0])[:-1]

In [22]:
predict(model, '46 * x + 87 - 400')

/var/folders/k7/dvm_qjfs0wd10b19j7jnj8rw0000gn/T/ipykernel_19988/3076454189.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((1, MAX_EQUATION_LENGTH, N_FEATURES), dtype=np.bool)


'                                      0'